In [99]:
### A wonderful tutorial on VariantAnnotation:
### https://www.bioconductor.org/packages/devel/bioc/vignettes/VariantAnnotation/inst/doc/VariantAnnotation.html#variant-call-format-vcf-files

In [104]:
library(VariantAnnotation)
library(dplyr)
library(TxDb.Hsapiens.UCSC.hg19.knownGene)
library(org.Hs.eg.db)
library(BSgenome.Hsapiens.UCSC.hg19)

In [100]:
vcf = readVcf("./Z.variantCall.SNPs.vcf", "hg19")

In [105]:
# Select variants on chr22, and modify the seqlevels to chr22 to match txdb seqlevels
vcf_chr22 = subset(vcf, seqnames(rowRanges(vcf)) == "22")
vcf_chr22 = keepSeqlevels(vcf_chr22, value = "22", pruning.mode = "coarse")
seqlevels(vcf_chr22) = "chr22"

In [123]:
##################################################################################

########## Criterion 1: all variants ##########

# Assign variants to the genes
txdb = TxDb.Hsapiens.UCSC.hg19.knownGene
loc = locateVariants(vcf_chr22, txdb, AllVariants())

#splt = split(loc$QUERYID, loc$GENEID)
#head(sapply(splt, function(x) length(unique(x))), 50)
#splt = split(loc$GENEID, loc$QUERYID)
#table(sapply(splt, function(x) length(unique(x)) > 1))

# Count mutations in genes
splt = split(loc$QUERYID, loc$GENEID)
variants = sapply(splt, function(x) length(unique(x)))
gene_ids = names(variants)

Warning message in valid.GenomicRanges.seqinfo(x, suggest.trim = TRUE):
“GRanges object contains 1892 out-of-bound ranges located on sequences
  74442, 73481, 74448, 73495, 73496, 73498, 73504, 74469, 74470, 74507,
  74508, 74509, 74510, 74505, 74506, 73534, 73543, 73547, 74544, 73574,
  73575, 73576, 74549, 74569, 74570, 73622, 73623, 73624, 73625, 74578,
  74579, 73648, 73649, 74600, 74603, 73666, 73667, 73671, 73672, 73673,
  73674, 73675, 73679, 74623, 74624, 74625, 74626, 74627, 74628, 74629,
  74630, 73700, 73701, 73703, 74655, 73713, 73714, 74662, 73735, 73736,
  73737, 73739, 73745, 73746, 73747, 73750, 73751, 73756, 73757, 73758,
  73762, 73763, 74705, 74706, 73776, 73777, 73782, 73793, 73798, 73801,
  73802, 73803, 73804, 73805, 73806, 74714, 74717, 74718, 74715, 74716,
  74719, 74720, 74722, 74723, 74724, 74725, 74742, 73830, 73831, 74756,
  74757, 74762, 74763, 74764, 73852, 74779, 74781, 73872, 73873, 73874,
  73878, 73879, 74800, 74801, 74802, 74803, 74809, 73926, 73929, 

In [116]:
########## Criterion 2: non-synonymous variants ##########

# Predict the coding effect of the variants
coding = predictCoding(vcf_chr22, txdb, seqSource=Hsapiens)

# Select the non-synoymous mutations (change AA)
nonsyn = coding[coding$REFAA != coding$VARAA, ]
variants = table(nonsyn$GENEID)
gene_ids = names(variants)
variants = as.numeric(variants)
names(variants) = gene_ids

Warning message in valid.GenomicRanges.seqinfo(x, suggest.trim = TRUE):
“GRanges object contains 1892 out-of-bound ranges located on sequences
  74442, 73481, 74448, 73495, 73496, 73498, 73504, 74469, 74470, 74507,
  74508, 74509, 74510, 74505, 74506, 73534, 73543, 73547, 74544, 73574,
  73575, 73576, 74549, 74569, 74570, 73622, 73623, 73624, 73625, 74578,
  74579, 73648, 73649, 74600, 74603, 73666, 73667, 73671, 73672, 73673,
  73674, 73675, 73679, 74623, 74624, 74625, 74626, 74627, 74628, 74629,
  74630, 73700, 73701, 73703, 74655, 73713, 73714, 74662, 73735, 73736,
  73737, 73739, 73745, 73746, 73747, 73750, 73751, 73756, 73757, 73758,
  73762, 73763, 74705, 74706, 73776, 73777, 73782, 73793, 73798, 73801,
  73802, 73803, 73804, 73805, 73806, 74714, 74717, 74718, 74715, 74716,
  74719, 74720, 74722, 74723, 74724, 74725, 74742, 73830, 73831, 74756,
  74757, 74762, 74763, 74764, 73852, 74779, 74781, 73872, 73873, 73874,
  73878, 73879, 74800, 74801, 74802, 74803, 74809, 73926, 73929, 

In [124]:
##################################################################################

# Convert EntrezID into Symbol
symbols = mapIds(org.Hs.eg.db,
                  keys = gene_ids,
                  column = "SYMBOL",
                  keytype = "ENTREZID",
                  multiVals = "first")

df = data.frame(row.names = gene_ids, Mutation_count = variants, Gene_symbol = symbols)

'select()' returned 1:1 mapping between keys and columns



In [125]:
# Locate the genes
gene_locs = genes(txdb, filter = list(gene_id = gene_ids), single.strand.genes.only=T)
gene_locs$symbol = symbols[match(gene_locs$gene_id, gene_ids)]
df_ft = data.frame(
    row.names = gene_locs$gene_id,
    Chromosome = as.character(seqnames(gene_locs)),
    Gene_symbol = gene_locs$symbol,
    stringsAsFactors = FALSE
)

  3 genes were dropped because they have exons located on both strands of
  the same reference sequence or on more than one reference sequence, so
  cannot be represented by a single genomic range.
  Use 'single.strand.genes.only=FALSE' to get all the genes in a
  GRangesList object, or use suppressMessages() to suppress this message.



In [127]:
# There is a potential issue here that if choosing IntronVariants() in the locateVariants function, 
# the identified genes are located in chromosome 17 and 18
df_ft %>% head(10)

,Chromosome,Gene_symbol
,<chr>,<chr>
1000,chr18,CDH2
100037417,chr22,DDTL
100101467,chr18,ZSCAN30
100126318,chr22,MIR301B
100128531,chr22,KIAA1671-AS1
100128893,chr18,GATA6-AS1
100128946,chr22,LINC01310
100130370,chr17,LINC03048
100130418,chr22,CECR7


In [119]:
# Filter genes not on chr22 and rank them by mutation counts
df$Chromosome = df_ft[rownames(df),'Chromosome']
df = df %>% filter(Chromosome == 'chr22') %>% arrange(desc(Mutation_count))
#write.csv(df,file='./genelist/genelist_AllVariants.csv')
write.csv(df,file='./genelist/genelist_NonSynoymousVariants.csv')

In [120]:
# Select top10 genes, get their location and subset the VCF file
g10 = rownames(df)[1:10]
g10_locs = gene_locs[names(gene_locs) %in% g10]
vcf_ranges = rowRanges(vcf_chr22)
hits = findOverlaps(vcf_ranges, g10_locs)
vcf_g10 = vcf_chr22[queryHits(hits)]
#writeVcf(vcf_g10, "./genelist/gene_variants_chr22_AllVariants.vcf")
writeVcf(vcf_g10, "./genelist/gene_variants_chr22_NonSynoymousVariants.vcf")